In [111]:
import glob
import os
import sys
import pandas as pd
import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from numbers import Number
from pprint import pprint
import logging
import operator

pd.options.display.max_rows = 10

In [112]:
en_file = './Accounts/q3q4_output/*.csv'

keywords_chosen = '13_FSDS_Goals_Keywords_EN.csv'

OUTPUT_COLS = ['id', 'num_comments', 'num_shares',
               'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads',
                'num_special', 'num_wows', 'Reactions_SUM', 
               'FSDS_matched_keywords', 'FSDS_category']

In [113]:
# read all accounts csv files and save chosen columns to dataframe
filePaths = glob.glob(en_file)
data_df = pd.DataFrame(columns=OUTPUT_COLS)
for filename in filePaths:
    #print(filename)    
    df_i = pd.read_csv(filename, encoding = 'utf-8')
    data_df = data_df.append(df_i, ignore_index=True)

data_df = data_df[OUTPUT_COLS]
data_df

,id,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,FSDS_category
0,318424514044_10154848488474045,49,353,0,1,3005,67,0,0,6,3481,[],unknown
1,318424514044_10155396901694045,695,129,25,7,214,48,11,0,10,1139,[],unknown
2,318424514044_10154830856069045,20,81,0,0,929,63,0,0,4,1097,[],unknown
3,318424514044_10154700497599045,47,142,1,1,612,14,0,0,3,820,"['development', 'development']",Clean growth
4,318424514044_10154771845659045,28,151,0,14,603,16,0,0,1,813,"['development', 'development']",Clean growth
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46814,2017-10-04_08-22-12,0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,3,[],unknown
46815,2016-12-08_08-45-08,0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,3,"['infrastructure', 'gas', 'vehicle', 'electric...",Low-carbon government
46816,2016-12-08_08-20-07,0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,2,['economy'],Clean growth
46817,2016-12-08_08-17-30,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,"['mountain', 'economy']",Clean growth


In [114]:
# set of punctuations to remove from text
exclude = set(string.punctuation)

stopWords = set(stopwords.words('english'))

lemma = WordNetLemmatizer()    # NLTK English lemmatizer

# detect_lang function can be use to check the percentage of non English posts
# note that missing value NaN can be detected as many different languages such english, spanish or italian
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        return 'error'
    return lang

# lemmatize_keywords also clears 'nan' from input keyword list file
# lemmatization is conducted based on context, some words may not get lemmatized, 
# e.g. "local eating" does not get lemmatized to "local eat"
def lemmatize_keywords(col):
    if str(col).lower() == 'nan':
        return ''
    return '_'.join(lemma.lemmatize(word).lower() for word in col.replace('’', '\'').replace('.', '').split()) #


In [115]:
# load keywords list

keywords_df = pd.read_csv(keywords_chosen, encoding='latin-1')   # "ISO-8859-1"
KEYWORDS_COLS = keywords_df.columns
lemma_keywords_df = pd.DataFrame(columns=KEYWORDS_COLS)
category_dict = {}
keywords_list = set()
for col in KEYWORDS_COLS:
    lemma_keywords_df[col] = keywords_df[col].astype(str).apply(lemmatize_keywords)
    category_dict[col.lower()] = set(lemma_keywords_df[col].tolist())
    category_dict[col.lower()].remove('')
    keywords_list = keywords_list.union(category_dict[col.lower()])

# the following is meant to clean a column name
try:
    category_dict['connecting canadians with nature'] = category_dict['connecting canadians with nature ']
    del category_dict['connecting canadians with nature ']
except:
    pass

In [116]:
category_dict

{'clean drinking water': {'accessible',
  'affordable',
  'aquifer',
  'boil_water',
  'clean',
  'drinking',
  'dumping',
  'first_nation',
  'flow',
  'forum_for_leadership_on_water',
  'fresh_water',
  'freshwater',
  'health',
  'hygiene',
  'indigenous_land',
  'lake',
  'municipality',
  'on-reserve',
  'phoshporus',
  'public_health',
  'river',
  'sanitation',
  'untreated_wastewater',
  'wastewater',
  'wastewater_treatment',
  'water',
  'water-related',
  'water-use',
  'water_advisory',
  'water_contamination',
  'water_harvesting',
  'water_infrastructure',
  'water_pollution',
  'water_resource_management',
  'water_scarcity',
  'water_supply',
  'water_system',
  'wetland'},
 'clean energy': {'clean_energy',
  'clean_power_generation',
  'clean_technology',
  'cleaner_fossil-fuel',
  'cleanest_electricity_system',
  'coalfired_electricity',
  'diesel',
  'electricity',
  'electricity_generation',
  'energy',
  'energy_efficiency',
  'energy_solution',
  'gas',
  'global_

In [117]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.lower()

In [118]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('unknown', 'other')
data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('connecting canadians with nature ', 'connecting canadians with nature')

In [119]:
def find_goal_words_only(row):
    matched_words = row['FSDS_matched_keywords'].replace('\', \'', ',').replace('[\'', '').replace('\']', '').replace('[]', '')
    if matched_words == '':
        return ''
    #print('Processing FSDS Goal: ' + row['FSDS_category'])
    matched_word_list = matched_words.split(',')
    matched_word_set = set(matched_word_list)   #### this lines removes duplicated matched words
    remove_words = []
    for word in matched_word_set:
        if word not in category_dict[row['FSDS_category']]:            
            remove_words.append(word)
            #print('This word does not belong to this category: ' + word)
    for word in remove_words:
        matched_word_set.remove(word)
    return str(', '.join(sorted(list(matched_word_set), key=str.lower)))

In [120]:
data_df['matched_goal_words_only'] = data_df.apply(find_goal_words_only, axis=1)
data_df

,id,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,FSDS_category,matched_goal_words_only
0,318424514044_10154848488474045,49,353,0,1,3005,67,0,0,6,3481,[],other,
1,318424514044_10155396901694045,695,129,25,7,214,48,11,0,10,1139,[],other,
2,318424514044_10154830856069045,20,81,0,0,929,63,0,0,4,1097,[],other,
3,318424514044_10154700497599045,47,142,1,1,612,14,0,0,3,820,"['development', 'development']",clean growth,development
4,318424514044_10154771845659045,28,151,0,14,603,16,0,0,1,813,"['development', 'development']",clean growth,development
5,318424514044_10154849106479045,25,62,0,0,508,23,0,0,0,618,"['development', 'development']",clean growth,development
6,318424514044_10154702444009045,305,86,1,0,187,6,0,0,4,589,[],other,
7,318424514044_10154732005664045,17,66,0,3,463,9,0,0,0,558,"['development', 'development']",clean growth,development
8,318424514044_10154491555714045,2,93,0,0,399,4,0,0,0,498,"['specie', 'specie']",healthy wildlife populations,specie
9,318424514044_10156589291004045,13,280,1,0,152,18,1,0,5,470,[],other,


In [122]:
f = {'id': ['count'], 'num_comments': ['sum'], 'num_shares': ['sum'], 'num_angrys': ['sum'], 'num_hahas': ['sum'], 
     'num_likes': ['sum'], 'num_loves': ['sum'], 'num_sads': ['sum'], 'num_special': ['sum'], 'num_wows': ['sum'], 
     'Reactions_SUM': ['sum'], 'FSDS_matched_keywords':[lambda col: ', '.join(col)], 
     'matched_goal_words_only':[lambda col: ', '.join(col)],}
output_df = data_df.groupby(['FSDS_category']).agg(f).reset_index()
output_df.columns = ['FSDS_category', 'id_count', 'num_comments', 'num_shares', 'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads', 'num_special', 'num_wows', 'Reactions_SUM', 'FSDS_matched_keywords', 'matched_goal_words_only', ]
output_df

,FSDS_category,id_count,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,matched_goal_words_only
0,clean drinking water,1857,1375,25952,6,12,72465,1654,22,3,502,101991,"['water', 'water', 'water'], ['drinking'], ['f...","water, drinking, freshwater, water, clean, hea..."
1,clean energy,414,314,6986,10,5,3336,56,1,1,20,10729,"['gas', 'park'], ['energy', 'energy', 'oil', '...","gas, energy, oil, diesel, electricity, heat, r..."
2,clean growth,1416,605,16071,13,45,18836,449,7,5,111,36142,"['development', 'development'], ['development'...","development, development, development, develop..."
3,connecting canadians with nature,1303,22525,52085,23,266,917235,16596,151,36,3802,1012719,"['nature'], ['nature'], ['conservation', 'cons...","nature, nature, conservation, nature, park, pa..."
4,effective action on climate change,526,391,9701,9,7,6858,88,8,-5,11,17068,"['climate_change', 'water', 'water', 'water', ...","climate_change, ecosystem, global_warming, cli..."
5,healthy coasts and oceans,3041,1531,44599,18,23,76409,1234,23,1,374,124212,"['sea', 'ecosystem', 'ecosystem', 'ecosystem',...","ecosystem, sea, fish, shellfish, plant, water,..."
6,healthy wildlife populations,2623,4741,48591,42,152,209295,5699,266,18,1402,270206,"['specie', 'specie'], ['wildlife', 'wildlife',...","specie, specie, wildlife, wildlife, conservati..."
7,low-carbon government,2782,2889,24743,632,882,24520,304,56,2,135,54163,"['energy', 'energy'], ['equipment'], ['energy'...","energy, equipment, energy, energy, emission, e..."
8,modern and resilient infrastructure,1386,431,10756,10,35,11409,226,20,-3,54,22938,"['research', 'lake', 'lake'], ['research', 're...","research, facility, research, facility, innova..."
9,other,28152,16387,522513,662,912,423362,13061,604,279,3232,981012,"[], [], [], [], [], [], [], [], [], [], [], []...",", , , , , , , , , , , , , , , , , , , , , , , ..."


In [126]:
def check_flag(col, word):
    word_set = set(col.split(', '))
    if word in word_set:
        return 'Y'
    else:
        return 'N'   
    
def find_top_topics(row):
    print('\n' + 'Processing FSDS Goal: ' + row['FSDS_category'])
    matched_words_list = row['matched_goal_words_only'].split(', ')
    #print(matched_words_list)
    counter = {}
    for word in set(matched_words_list):
        counter[word] = 0
    for word in matched_words_list:
        counter[word] += 1
    top_topics = sorted(counter, key=counter.get, reverse=True)  
    #print(counter)
    result = ''
    temp_df = data_df[data_df['FSDS_category']==row['FSDS_category']]
    for i in range(0, 15):
        try:
            top_topics[i]
        except:
            return result
        temp_df['flag'] = 'N'
        temp_df['flag'] = temp_df['matched_goal_words_only'].apply(check_flag, word=top_topics[i])
        react_sum = temp_df[temp_df['flag'] == 'Y']['Reactions_SUM'].sum()
        result += top_topics[i].replace('specie', 'species').replace('_', ' ') + ': ' + str(react_sum) + '\n'  
    return result

output_df['Topics/Issues'] = output_df.apply(find_top_topics, axis=1)
output_df


Processing FSDS Goal: clean drinking water


/Users/chao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Processing FSDS Goal: clean energy

Processing FSDS Goal: clean growth

Processing FSDS Goal: connecting canadians with nature

Processing FSDS Goal: effective action on climate change

Processing FSDS Goal: healthy coasts and oceans

Processing FSDS Goal: healthy wildlife populations

Processing FSDS Goal: low-carbon government

Processing FSDS Goal: modern and resilient infrastructure

Processing FSDS Goal: other

Processing FSDS Goal: pristine lakes and rivers

Processing FSDS Goal: safe and healthy communities

Processing FSDS Goal: sustainable food

Processing FSDS Goal: sustainably managed lands and forests


,FSDS_category,id_count,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,matched_goal_words_only,Topics/Issues
0,clean drinking water,1857,1375,25952,6,12,72465,1654,22,3,502,101991,"['water', 'water', 'water'], ['drinking'], ['f...","water, drinking, freshwater, water, clean, hea...",water: 48601\nriver: 20780\nlake: 39172\nhealt...
1,clean energy,414,314,6986,10,5,3336,56,1,1,20,10729,"['gas', 'park'], ['energy', 'energy', 'oil', '...","gas, energy, oil, diesel, electricity, heat, r...",oil: 4448\nheat: 1704\ngas: 2977\nenergy: 690\...
2,clean growth,1416,605,16071,13,45,18836,449,7,5,111,36142,"['development', 'development'], ['development'...","development, development, development, develop...",marine: 22032\neconomy: 6223\ngrowth: 2821\nde...
3,connecting canadians with nature,1303,22525,52085,23,266,917235,16596,151,36,3802,1012719,"['nature'], ['nature'], ['conservation', 'cons...","nature, nature, conservation, nature, park, pa...",park: 910948\nnature: 282407\nengagement: 4181...
4,effective action on climate change,526,391,9701,9,7,6858,88,8,-5,11,17068,"['climate_change', 'water', 'water', 'water', ...","climate_change, ecosystem, global_warming, cli...",climate change: 5332\ncommitment: 7069\necosys...
5,healthy coasts and oceans,3041,1531,44599,18,23,76409,1234,23,1,374,124212,"['sea', 'ecosystem', 'ecosystem', 'ecosystem',...","ecosystem, sea, fish, shellfish, plant, water,...",coast: 53396\nocean: 35616\nfishery: 7951\nsea...
6,healthy wildlife populations,2623,4741,48591,42,152,209295,5699,266,18,1402,270206,"['specie', 'specie'], ['wildlife', 'wildlife',...","specie, specie, wildlife, wildlife, conservati...",species: 41215\nwildlife: 75679\nconservation:...
7,low-carbon government,2782,2889,24743,632,882,24520,304,56,2,135,54163,"['energy', 'energy'], ['equipment'], ['energy'...","energy, equipment, energy, energy, emission, e...",energy: 17720\noperation: 5006\nvehicle: 9475\...
8,modern and resilient infrastructure,1386,431,10756,10,35,11409,226,20,-3,54,22938,"['research', 'lake', 'lake'], ['research', 're...","research, facility, research, facility, innova...",research: 13022\nindustry: 3489\ninnovation: 3...
9,other,28152,16387,522513,662,912,423362,13061,604,279,3232,981012,"[], [], [], [], [], [], [], [], [], [], [], []...",", , , , , , , , , , , , , , , , , , , , , , , ...",: 981012\n


In [127]:
output_df = output_df[['FSDS_category', 'Reactions_SUM', 'Topics/Issues']]
output_df.columns = ['FSDS Goal', 'Reaction_Sum', 'Topics/Issues']
output_df['FSDS Goal'] = output_df['FSDS Goal'].str.title()
output_df = output_df.sort_values('Reaction_Sum', ascending=False)
output_df.to_csv('./section3-3_table_en.csv', index=None)
output_df

/Users/chao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,FSDS Goal,Reaction_Sum,Topics/Issues
3,Connecting Canadians With Nature,1012719,park: 910948\nnature: 282407\nengagement: 4181...
9,Other,981012,: 981012\n
6,Healthy Wildlife Populations,270206,species: 41215\nwildlife: 75679\nconservation:...
13,Sustainably Managed Lands And Forests,234187,forest: 47655\nland: 28601\nmountain: 156261\n...
5,Healthy Coasts And Oceans,124212,coast: 53396\nocean: 35616\nfishery: 7951\nsea...
0,Clean Drinking Water,101991,water: 48601\nriver: 20780\nlake: 39172\nhealt...
11,Safe And Healthy Communities,69642,community: 18158\nice: 28660\ncost: 3924\ncity...
7,Low-Carbon Government,54163,energy: 17720\noperation: 5006\nvehicle: 9475\...
2,Clean Growth,36142,marine: 22032\neconomy: 6223\ngrowth: 2821\nde...
12,Sustainable Food,24164,food: 5762\nfeed: 16288\naquaculture: 790\nflo...
